In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix, ndcs_to_pixels, grid_to_ndc_pcl_linear_view
from visualization import load_depth
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
from voxelmaps import camera_to_pointcloud, load_scene_db_data, get_main_image_name, scene_to_pointcloud, scene_to_voxelmap, NoMainImageException, scene_to_voxelmap_with_map, get_main_image, ndc_pcl_to_grid_linear_view, generate_frustum_points
import voxelmaps
import time
import csv
import random
import gta_math

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file
visualization.use_cache = False

conn = visualization.get_connection()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]
out_root_directory = r'D:\\'


# setting linear samping of voxelmap in view space
linear_view_sampling = True # otherwise, it is linear in NDC depth -> hyperbolic in view
gta_math.PROJECTING = True


In [3]:
run_id = 6

cur.execute("""SELECT DISTINCT scene_id \
  FROM snapshots \
  WHERE run_id = {} \
  """.format(run_id))

scenes = []
for row in cur:
    res = dict(row)
    scenes.append(res)

image_names = {}

print('There are {} scenes'.format(len(scenes)))


There are 8438 scenes


### functions for each scene

In [4]:
# import time
voxelmaps.MAX_DISTANCE = 30 
# I will create frustum up to 25 meters, so I take distance to 30 meters
# in meters, meaning I care only for point up to 25 meters, 
# in depth for 100 bins, it corresponds to 25 centimers big voxels

points_cache = {}

def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def convert_tiff(in_directory, out_directory, out_name, name):
    # throws OSError
    out_format = 'jpg'
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    if os.path.exists(outfile):
        return outfile
    infile = os.path.join(in_directory, name)
    im = Image.open(infile)
    im = im.convert(mode="RGB")
    # print("Generating new format for {} to new file {}".format(name, out_name))
    im.save(outfile)
    return outfile

def convert_scene_to_ndc_voxelmap(cameras):
    # this method is just fucking slow, because of scene_to_voxelmap_with_map
    # here I generate points cuboid which will be output of NN.
    # these points will be transferred into camera view frustum
    # for this frustum, I take voxel values of calculated voxelmap
    # I use these voxel values as values for cuboid in NDC, the NN output
    #start = time.time()

    voxels, values, map_voxel_size, map_obj = scene_to_voxelmap_with_map(cameras, subsampling_size=0.12)

    #end = time.time()
    #print('scene to whole voxelmap:', end - start)
    #start = time.time()

    cam = get_main_image(cameras)
    proj_matrix = cam['proj_matrix']
    view_matrix = cam['view_matrix']

    #end = time.time()
    #print('getting main cam and stuff:', end - start)
    #start = time.time()

    x_range = 160  # X output size of NN
    y_range = 120  # Y output size of NN
    z_range = 100  # Z output size of NN
    z_meters_min = 1.5
    z_meters_max = 25
    positions, z_max, z_min = generate_frustum_points(proj_matrix, x_range, y_range, z_range, z_meters_min, z_meters_max, linear_view_sampling)

    #end = time.time()
    #print('generating frustum points:', end - start)
    #start = time.time()

    points_view = ndc_to_view(positions, proj_matrix)
    points_world = view_to_world(points_view, view_matrix)

    #end = time.time()
    #print('obtaining world frustum:', end - start)
    #start = time.time()

    # here I find corresponding voxels for generated points, by obtaining voxelmap reference
    voxel_values = map_obj.get_voxels(points_world[0:3, :], np.zeros((points_world.shape[1], 1)))
    occupied_ndc_points = positions[:, voxel_values > 0]
    invalid_ndc_points = positions[:, np.isnan(voxel_values)]

    #end = time.time()
    #print('obtaining voxelmap in NDC:', end - start)
    #start = time.time()
    # this converts ndc pointcloud to bool grid
    occupied_ndc_grid = ndc_pcl_to_grid_linear_view(x_range, y_range, z_range, occupied_ndc_points, proj_matrix, z_meters_min, z_meters_max)
    invalid_ndc_grid = ndc_pcl_to_grid_linear_view(x_range, y_range, z_range, invalid_ndc_points, proj_matrix, z_meters_min, z_meters_max)
    
    # merging obstacles and invalids together, having
    # obstacle = 1
    # free = 0
    # unknown = -1
    return occupied_ndc_grid - invalid_ndc_grid


def convert_scene_to_img_and_voxelmap(in_directory, out_directory, scene_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    # start = time.time()
    cameras = load_scene_db_data(scene_id)
    try:
        image_name = get_main_image_name(cameras)
        rgb_outfile = convert_tiff(in_directory, out_directory, image_name, image_name+'.tiff')
        outfile = os.path.join(out_directory, "{}.npy".format(image_name))

        image_names[scene_id] = (rgb_outfile, outfile)
        if os.path.exists(outfile):
            return
        
        #start = time.time()
        occupied_ndc_positions = convert_scene_to_ndc_voxelmap(cameras)
        #end = time.time()
        #print('time to do whole scene to voxelmap:', end - start)
        #start = time.time()
        
        #occupied_ndc_positions.tofile(outfile)
        np.save(outfile, occupied_ndc_positions)
        #end = time.time()
        #print('time to save:', end - start)
        
    except (OSError, NoMainImageException) as e:
        print('No main image for scene {}, skipping.'.format(scene_id))


### actually running the extraction

In [ ]:
workers = 10

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(scenes)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(convert_scene_to_img_and_voxelmap)(in_directory, out_directory, i['scene_id']) for i in scenes)


N/A% 0 |                                                            |   0.0 s/BC:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in greater
  2% 203 |#                                                         |   0.2 B/s

No main image for scene 592377fd-866a-4d81-bdb0-4e63ae584bab, skipping.


  2% 212 |#                                                         |   0.2 B/sC:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:2642: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
  2% 228 |#                                                         |   0.2 B/s

No main image for scene c70fe8d1-e628-483e-a27a-6a2c7b862857, skipping.


  7% 644 |####                                                      |   0.2 B/s

...

 11% 993 |######                                                    |   0.2 B/s

No main image for scene 08056d94-ab3d-42cd-8000-f9828444e783, skipping.


 12% 1033 |######                                                   |   0.2 B/s

No main image for scene 6f634b01-f5a0-4d65-b2fc-7df58b0d8904, skipping.


 12% 1036 |######                                                   |   0.2 B/s

### generating filenames list

In [6]:
filenames = []
for scene_id, (rgb_outfile, outfile) in image_names.items():
    rgb_name = os.path.relpath(os.path.join(out_directory, rgb_outfile), start=out_root_directory)
    rgb_name = rgb_name.replace('\\', '/')
    voxelmap_name = os.path.relpath(os.path.join(out_directory, outfile), start=out_root_directory)
    voxelmap_name = voxelmap_name.replace('\\', '/')
    filenames.append([rgb_name, voxelmap_name])

    # r je abych nemusel psát zpětná lomítka
with open(os.path.join(out_root_directory, 'whole-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(filenames)

random.shuffle(filenames)

train_ratio = 0.8
train_threshold = int(train_ratio * len(image_names))
train_filenames = filenames[:train_threshold]
test_filenames = filenames[train_threshold:]

with open(os.path.join(out_root_directory, 'train-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(train_filenames)
with open(os.path.join(out_root_directory, 'test-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(test_filenames)


### running extraction for one scene

In [11]:
convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[20]['scene_id'])

In [ ]:
# just some timing experiments
for i in range(5):
    convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[i]['scene_id'])

In [ ]:
prev = None
for i in range(5):
    cameras = load_scene_db_data(scenes[i]['scene_id'])
    cam = get_main_image(cameras)
    proj_matrix = cam['proj_matrix']
    if prev is not None:
        print(np.array_equal(prev, proj_matrix))
    prev = proj_matrix

In [ ]:
prev

In [12]:
loaded_values = np.load(r'D:\ml-datasets-voxel\2018-03-07--16-38-30--192.npy')
np.unique(loaded_values)

array([-1,  0,  1], dtype=int8)